In [35]:
import os
import sys
sys.path.append('./')
sys.path.append('../')
import numpy as np

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from art.estimators.classification import SklearnClassifier
from art.attacks.evasion import FastGradientMethod

from Core.dataset import CDataset

# Step 1: Load dataset

## Wine Dataset

In [ ]:
data = load_wine()
X, y = data.data, data.target

# Standardize features for stability
scaler = StandardScaler()
X = scaler.fit_transform(X)

## Kyoto

In [23]:
cDATA = 'Kyoto'

objDS = CDataset()

strFileName =  r'../../DATA/Kyoto2015DS.csv'
#strPath = os.path.join(strRoot, strFileName)
X, y = objDS.GetKyotoDataset(strFileName)
print('Kyoto:', X.shape, y.shape)

fTestSize = 0.9

CDataset Object Created
label
0    30000
1    30000
dtype: int64
Kyoto: (60000, 13) (60000, 1)


## Spilt Data

In [24]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=fTestSize, 
                                                    random_state=42)

In [39]:
print('Training Data:', X_train.shape, y_train.shape)
print('Testing Data:', X_test.shape, y_test.shape)

print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))

Training Data: (6000, 13) (6000, 1)
Testing Data: (54000, 13) (54000, 1)
(array([0, 1], dtype=int64), array([3082, 2918], dtype=int64))
(array([0, 1], dtype=int64), array([26918, 27082], dtype=int64))


# Step 2: Train a simple classifier

## Train

In [45]:
model = RandomForestClassifier()
#model = SVC(C=1.0, kernel="rbf")
model.fit(X_train, y_train)

C:\Users\admin\AppData\Local\Temp\ipykernel_10056\1884825294.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


RandomForestClassifier()

## Test

In [46]:
# Evaluate on clean test set
acc_clean = model.score(X_test, y_test)
print(f"Accuracy on clean test set: {acc_clean:.4f}")

Accuracy on clean test set: 0.9708


# Step 3: Wrap model for ART

In [47]:
classifier = SklearnClassifier(model=model)
classifier.fit(X_train, y_train)

c:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)
c:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


ValueError: nb_classes must be greater than or equal to 2.

# Step 4: Create adversarial attack (FGSM)

In [ ]:
attack = FastGradientMethod(estimator=classifier, eps=0.1)

# Step 5: Generate adversarial examples

In [ ]:
X_test_adv = attack.generate(x=X_test)

# Step 6: Evaluate on adversarial set

In [ ]:
acc_adv = model.score(X_test_adv, y_test)
print(f"Accuracy on adversarial test set: {acc_adv:.4f}")

# Optional: Compare a few original vs adversarial samples
print("\nOriginal sample (first row):", X_test[0])
print("Adversarial sample (first row):", X_test_adv[0])